# INTRODUCTION

There are lot of companies that provide the same service or sell the same products around the same areas and that has some disadvantage and advantages, so some of the advantages is that the customer will have options and the companies will keep the standard up just to stand out of their competitors. As a new business person with a new brand, competition can kill your business as people tend to go for what they are used to. So it becomes very important to locate your business where there are few business that provide the same service and also to check the target market, i.e. knowing where your competitors are located is a very important factor as it will guide  you  to locate yours .In this project we are going to search for a location where there are few competitors, preferably coffee shops so that it can help a new business to be situated where there’s less competition.

# DATA SECTION

To accomplish this we going to get the data of all the businesses in Toronto and perform the analysis on them. We going to examine the types of the businesses in Toronto and filter out the fast food shops and perform analysis to see the ones that are trending together with the one good ratings, in order to be able to identify the strong and the weak competitors.With the above being done,The analysis will continue to figure out if there are strong competitors located close to each other or not, this will help us unpack the notion of locating the business where there are few strong competitors . Moreover, then we going to get the data of two of the nearest venues then predict the next percentages that next  customer will come to our  new shop (i.e. not return to our competitor).Assumption would be that if the customer is not returning to the old shop then the customer is coming to the new shop, the prediction will be done using the Decision tree, SVM,KNN and logistic. Note that all the data will be from Foursquare location and https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import urllib.request

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.65 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.29 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.77 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.06 MB/s
Libraries imported.


In [2]:
wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page=urllib.request.urlopen(wiki)

from bs4 import BeautifulSoup

soup = BeautifulSoup(page)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [3]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],

In [4]:

all_tables=soup.find_all('table')

right_table=soup.find('table',class_="wikitable sortable")

In [5]:

A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
  
    cells = row.findAll('td')
    if len((cells))==3:
       
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [6]:
df=pd.DataFrame()
df['PostalCode']=A
df['BOROUGH']=B
df['NEIBOURHOOD']=C

In [7]:
df.drop(df[df.BOROUGH=='Not assigned'].index,inplace=True)
df['NEIBOURHOOD']=np.where(df['NEIBOURHOOD']=='Not assigned',df['BOROUGH'],df['NEIBOURHOOD'])
df2=df.groupby(['PostalCode','BOROUGH'])['NEIBOURHOOD'].apply(', '.join).str.replace('(\\n)', '').reset_index()

In [8]:
!wget -q -O can_geo.csv https://cocl.us/Geospatial_data;

In [9]:
df1= pd.read_csv("can_geo.csv")
df1=df1.rename(columns = {'Postal Code':'PostalCode'})

In [10]:
df3 = pd.merge(df2,df1)
df3.head()

,PostalCode,BOROUGH,NEIBOURHOOD,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
df4=df3[df3['BOROUGH'].str.contains('Toronto')]
df4.head()

,PostalCode,BOROUGH,NEIBOURHOOD,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [12]:
ToroMap = folium.Map(location=[43.6532, -79.3832], zoom_start=12)


for lat, lng, BOROUGH, NEIBOURHOOD in zip(df4['Latitude'], df4['Longitude'], df4['PostalCode'], df4['NEIBOURHOOD']):
    label = '{}, {}'.format(NEIBOURHOOD, BOROUGH)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(ToroMap)

ToroMap

In [68]:
CLIENT_ID = 'SXUPV3IPG1VRYAIEKCEIIXE4GR1ZGSUNIPTKXB5C02I514K0' # my Foursquare ID
CLIENT_SECRET = 'LC1PKVJZGO5RSMWKGYQL5JGSYC1FT14SW1OZM11FGB51GLBB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SXUPV3IPG1VRYAIEKCEIIXE4GR1ZGSUNIPTKXB5C02I514K0
CLIENT_SECRET:LC1PKVJZGO5RSMWKGYQL5JGSYC1FT14SW1OZM11FGB51GLBB


In [35]:
# retrieving top venues nearby all neighbourhoods in Toronto in 500m radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
# saving nearby venues into dataFrame
tor_venues = getNearbyVenues(names=df3['NEIBOURHOOD'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

In [37]:
print(tor_venues.shape)
tor_venues.head()

(2221, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [38]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [39]:
tor_venues['Venue Category']
df5=tor_venues[tor_venues['Venue Category'].str.contains('Coffee')]
df5

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
12,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop
23,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop
75,"L'Amoreaux West, Steeles West",43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop
101,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Aroma Espresso Bar,43.777700,-79.344652,Coffee Shop
110,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Starbucks,43.778287,-79.343331,Coffee Shop
119,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Tim Hortons,43.775249,-79.347740,Coffee Shop
124,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Tim Hortons,43.774965,-79.346285,Coffee Shop
166,Willowdale South,43.770120,-79.408493,Starbucks,43.768192,-79.413021,Coffee Shop
177,Willowdale South,43.770120,-79.408493,Starbucks,43.768627,-79.413100,Coffee Shop


In [19]:
df5.shape

(184, 7)

# The below map shows the location of all the coffee shops around Toronto

In [20]:
ToroMap = folium.Map(location=[43.6532, -79.3832], zoom_start=12)


for lat, lng, Venue, Neighborhood in zip(df5['Venue Latitude'], df5['Venue Longitude'], df5['Venue'], df5['Venue Category']):
    label = '{}, {}'.format(NEIBOURHOOD, BOROUGH)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(ToroMap)

ToroMap

# The below Table shows the count of each coffee shop branch


In [21]:
df5.groupby('Venue').count()       

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category
Venue,,,,,,
Aroma Espresso Bar,3,3,3,3,3,3
Arvo,1,1,1,1,1,1
Balzac's Coffee,2,2,2,2,2,2
Big Guy's Little Coffee Shop,1,1,1,1,1,1
Cafe Frappe,1,1,1,1,1,1
Coffee Island,2,2,2,2,2,2
Coffee Public,2,2,2,2,2,2
Coffee Time,1,1,1,1,1,1
Creeds Coffee Bar,1,1,1,1,1,1


# The below shows that we have 51 uniques coffee shops in Toronto

In [31]:

print('There are {} uniques Venue.'.format(len(df5['Venue'].unique())))

There are 51 uniques Venue.


# The coffee shops that have many branches means they are doing well in business and we dont want to locate our business next to them, hence you can locate the business anywhere apart from next to the below shops

Tim Hortons,
Starbucks
Nordstrom Ebar
Dineen @CommerceCourt